#**3. Insurance Penetration and Growth Potential Analysis**

In [2]:
import sys
import os

sys.path.append(os.path.abspath("D:/D26_Files/Phonepe_Analytics/Pulse_Case_Studies/"))

from db_connection import get_phonepe_engine


engine = get_phonepe_engine()

In [3]:
import pandas as pd
from sqlalchemy import text

In [4]:
tables_df = pd.read_sql("SHOW TABLES;", engine)
tables_df

,Tables_in_phonepe_db
0,agg_transaction
1,agg_user
2,insurance_transaction
3,map_transaction
4,map_user
5,top_transaction


In [5]:
Insurance_Transaction_df = pd.read_sql("SELECT * FROM insurance_transaction;", engine)
Insurance_Transaction_df.head()

,State,Year,Quarter,Insurance_txn_count,Insurance_txn_amount
0,Andaman & Nicobar Islands,2020,2,6,1360.0
1,Andaman & Nicobar Islands,2020,3,41,15380.0
2,Andaman & Nicobar Islands,2020,4,124,157975.0
3,Andaman & Nicobar Islands,2021,1,225,244266.0
4,Andaman & Nicobar Islands,2021,2,137,181504.0


In [13]:
Agg_User_df = pd.read_sql("SELECT * FROM agg_user;", engine)
Agg_User_df


,State,Year,Quarter,Registered_users,App_opens,Brand,Brand_count,Brand_percentage
0,Andaman & Nicobar Islands,2018,1,6740,0,Xiaomi,1665,0.247033
1,Andaman & Nicobar Islands,2018,1,6740,0,Samsung,1445,0.214392
2,Andaman & Nicobar Islands,2018,1,6740,0,Vivo,982,0.145697
3,Andaman & Nicobar Islands,2018,1,6740,0,Oppo,501,0.074332
4,Andaman & Nicobar Islands,2018,1,6740,0,OnePlus,332,0.049258
...,...,...,...,...,...,...,...,...
6727,West Bengal,2022,1,21919787,236131065,Lenovo,330017,0.015056
6728,West Bengal,2022,1,21919787,236131065,Infinix,284678,0.012987
6729,West Bengal,2022,1,21919787,236131065,Asus,280347,0.012790
6730,West Bengal,2022,1,21919787,236131065,Apple,277752,0.012671


*1.	Which states show the highest insurance adoption relative to PhonePe user base?*

In [34]:
insurance_adoption_df = pd.read_sql("""
SELECT 
i.State,
    SUM(i.Insurance_txn_count) as Total_Insurance_Transactions,
    MAX(u.Registered_users) as Total_Registered_Users,
    ROUND((SUM(i.Insurance_txn_count) * 100.0 / MAX(u.Registered_users)), 4) as Insurance_Adoption_Rate_Percentage
FROM insurance_transaction i
JOIN agg_user u 
    ON i.State = u.State 
    AND i.Year = u.Year 
    AND i.Quarter = u.Quarter   
GROUP BY i.State
HAVING MAX(u.Registered_users) > 0
ORDER BY Insurance_Adoption_Rate_Percentage DESC;
""", engine)
insurance_adoption_df.head()

,State,Total_Insurance_Transactions,Total_Registered_Users,Insurance_Adoption_Rate_Percentage
0,Andaman & Nicobar Islands,25168.0,76288,32.9908
1,Karnataka,4498054.0,31200349,14.4167
2,Goa,104148.0,738628,14.1002
3,Lakshadweep,770.0,5990,12.8548
4,Kerala,1082114.0,8972299,12.0606


*2.	Which states are lagging in insurance penetration despite high user registrations?*

In [32]:
lagging_insurance_df = pd.read_sql("""
SELECT 
i.State,
    MAX(u.Registered_users) as Total_Registered_Users,
    SUM(i.Insurance_txn_count) as Total_Insurance_Transactions,
    Round((SUM(i.Insurance_txn_count) * 100.0 / MAX(u.Registered_users)), 4) as Insurance_Adoption_Rate_Percentage
FROM insurance_transaction i
JOIN agg_user u 
    ON i.State = u.State 
    AND i.Year = u.Year 
    AND i.Quarter = u.Quarter
GROUP BY i.State
HAVING MAX(u.Registered_users) > 10000000
ORDER BY Total_Registered_Users DESC, Insurance_Adoption_Rate_Percentage ASC;
""", engine)
lagging_insurance_df

,State,Total_Registered_Users,Total_Insurance_Transactions,Insurance_Adoption_Rate_Percentage
0,Maharashtra,48227630,5556925.0,11.5223
1,Uttar Pradesh,38999692,2403137.0,6.1619
2,Karnataka,31200349,4498054.0,14.4167
3,Andhra Pradesh,23857951,2498562.0,10.4727
4,Rajasthan,23398857,1394063.0,5.9578
5,Telangana,22150612,2527580.0,11.4109
6,West Bengal,21919787,1661814.0,7.5813
7,Tamil Nadu,21667020,1756062.0,8.1048
8,Madhya Pradesh,19656147,1116412.0,5.6797
9,Gujarat,19265314,1256992.0,6.5246


*3.	How has insurance transaction volume & value grown quarter by quarter?*

In [9]:
quarterly_growth_df = pd.read_sql("""
SELECT 
Year,
Quarter,
    SUM(Insurance_txn_count) as Transaction_Volume,
    SUM(Insurance_txn_amount) as Transaction_Value
FROM insurance_transaction
GROUP BY Year, Quarter
ORDER BY Year, Quarter;
""", engine)
quarterly_growth_df

,Year,Quarter,Transaction_Volume,Transaction_Value
0,2020,2,185348.0,3.373217e+07
1,2020,3,354284.0,8.949508e+07
2,2020,4,248626.0,1.709799e+08
3,2021,1,318115.0,2.062957e+08
4,2021,2,363988.0,2.950654e+08
5,2021,3,374860.0,3.423888e+08
6,2021,4,526319.0,6.554201e+08
7,2022,1,617575.0,8.874138e+08
8,2022,2,630744.0,8.570128e+08
9,2022,3,806454.0,1.054673e+09


*4.	What is the distribution of insurance adoption between top vs bottom 10 states?*

In [35]:
top_10_df = pd.read_sql("""
SELECT 
i.State,
    MAX(u.Registered_users) as Total_Registered_Users,
    SUM(i.Insurance_txn_count) as Total_Insurance_Transactions,
    Round((SUM(i.Insurance_txn_count) * 100.0 / MAX(u.Registered_users)), 2) as Insurance_Adoption_Rate_Percentage
FROM insurance_transaction i
JOIN agg_user u 
    ON i.State = u.State 
    AND i.Year = u.Year 
    AND i.Quarter = u.Quarter
GROUP BY i.State
ORDER BY Insurance_Adoption_Rate_Percentage DESC
LIMIT 10;
""", engine)
top_10_df

,State,Total_Registered_Users,Total_Insurance_Transactions,Insurance_Adoption_Rate_Percentage
0,Andaman & Nicobar Islands,76288,25168.0,32.99
1,Karnataka,31200349,4498054.0,14.42
2,Goa,738628,104148.0,14.10
3,Lakshadweep,5990,770.0,12.85
4,Kerala,8972299,1082114.0,12.06
5,Maharashtra,48227630,5556925.0,11.52
6,Telangana,22150612,2527580.0,11.41
7,Delhi,13539197,1477355.0,10.91
8,Andhra Pradesh,23857951,2498562.0,10.47
9,Assam,4492589,455136.0,10.13


In [36]:
bottom_10_df = pd.read_sql("""
SELECT 
i.State,
    MAX(u.Registered_users) as Total_Registered_Users,
    SUM(i.Insurance_txn_count) as Total_Insurance_Transactions,
    Round((SUM(i.Insurance_txn_count) * 100.0 / MAX(u.Registered_users)), 2) as Insurance_Adoption_Rate_Percentage
FROM insurance_transaction i
JOIN agg_user u 
    ON i.State = u.State 
    AND i.Year = u.Year 
    AND i.Quarter = u.Quarter
GROUP BY i.State
ORDER BY Insurance_Adoption_Rate_Percentage ASC
LIMIT 10;
""", engine)
bottom_10_df

,State,Total_Registered_Users,Total_Insurance_Transactions,Insurance_Adoption_Rate_Percentage
0,Manipur,411900,10725.0,2.60
1,Nagaland,276483,9592.0,3.47
2,Sikkim,209616,10307.0,4.92
3,Punjab,7302726,371855.0,5.09
4,Arunachal Pradesh,335339,17710.0,5.28
5,Meghalaya,255839,13530.0,5.29
6,Himachal Pradesh,2143483,119988.0,5.60
7,Madhya Pradesh,19656147,1116412.0,5.68
8,Odisha,12124779,707597.0,5.84
9,Ladakh,92492,5445.0,5.89


*5.	Which states represent the biggest untapped opportunity for insurance expansion?*

In [38]:
untapped_opportunity_df = pd.read_sql("""
SELECT 
i.State,
    MAX(u.Registered_users) as Total_Registered_Users,
    SUM(u.App_opens) as App_Engagement,
    SUM(i.Insurance_txn_count) as Total_Insurance_Transactions,
    Round((SUM(i.Insurance_txn_count) * 100.0 / MAX(u.Registered_users)), 2) as Insurance_Adoption_Rate_Percentage,
    MAX(u.Registered_users) - SUM(i.Insurance_txn_count) as Untapped_Users
FROM insurance_transaction i
JOIN agg_user u 
    ON i.State = u.State 
    AND i.Year = u.Year 
    AND i.Quarter = u.Quarter
GROUP BY i.State
HAVING Total_Registered_Users > 1000000 AND Insurance_Adoption_Rate_Percentage < 10
ORDER BY Untapped_Users DESC, App_Engagement DESC;
""", engine)
untapped_opportunity_df

,State,Total_Registered_Users,App_Engagement,Total_Insurance_Transactions,Insurance_Adoption_Rate_Percentage,Untapped_Users
0,Uttar Pradesh,38999692,6.068443e+10,2403137.0,6.16,36596555.0
1,Rajasthan,23398857,8.984305e+10,1394063.0,5.96,22004794.0
2,West Bengal,21919787,2.668705e+10,1661814.0,7.58,20257973.0
3,Tamil Nadu,21667020,3.415250e+10,1756062.0,8.10,19910958.0
4,Madhya Pradesh,19656147,6.334795e+10,1116412.0,5.68,18539735.0
5,Gujarat,19265314,2.757634e+10,1256992.0,6.52,18008322.0
6,Bihar,18484658,3.206862e+10,1210616.0,6.55,17274042.0
7,Haryana,12866988,2.227934e+10,1119052.0,8.70,11747936.0
8,Odisha,12124779,3.270679e+10,707597.0,5.84,11417182.0
9,Punjab,7302726,7.830881e+09,371855.0,5.09,6930871.0
